In [1]:
# load dataset
import os

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print('device is ', device)

transcripts = {}

with open('dataset/transcripts.txt', 'r') as file:
    for line in file:
        filename, transcription = line.split(' ', maxsplit=1)
        transcripts[filename] = transcription.strip()

training_dataset = {}
testing_dataset = {}

# filter training and testing dataset
folder_path = 'dataset/Testing'

testing = os.listdir(folder_path)

from datasets import Audio, Dataset

for filename in testing:
    name, extension = os.path.splitext(filename)
    transcript = transcripts[name]
    testing_dataset[name] = transcript
    transcripts.pop(name)

training_dataset = transcripts

assert len(os.listdir('dataset/Training')) == len(training_dataset)
assert len(os.listdir('dataset/Testing')) == len(testing_dataset)


device is  cuda


In [2]:
# Convert to dataset
def convert_dataset(audio_dir, dataset_dict):
    filenames = list(dataset_dict.keys())
    scripts = list(dataset_dict.values())

    audio_paths = [f'{audio_dir}/{filen}.flac' for filen in filenames]
    dataset = Dataset.from_dict({
        "audio": audio_paths,
        "transcript": scripts,
        "filename": filenames
    })

    # Cast the audio column to Audio() type
    dataset = dataset.cast_column("audio", Audio(sampling_rate=16000))
    return dataset


training_dataset = convert_dataset('dataset/Training', training_dataset)
testing_dataset = convert_dataset('dataset/Testing', testing_dataset)

# print(training_dataset["audio"][0]['path'])
# print()

In [3]:
from transformers import WhisperTokenizer, WhisperFeatureExtractor

base_model = "openai/whisper-small"

feature_extractor = WhisperFeatureExtractor.from_pretrained(base_model)

tokenizer = WhisperTokenizer.from_pretrained(base_model, language="English", task="transcribe")


def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]

    # encode target text to label ids
    batch["labels"] = tokenizer(batch["transcript"]).input_ids
    return batch


training_dataset = training_dataset.map(prepare_dataset, num_proc=1)
testing_dataset = testing_dataset.map(prepare_dataset, num_proc=1)
print('complete mapping')


Map:   0%|          | 0/905 [00:00<?, ? examples/s]

Map:   0%|          | 0/223 [00:00<?, ? examples/s]

complete mapping


In [4]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
import torch
from typing import Any, Dict, List, Union


class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any
    decoder_start_token_id: int

    def __init__(self, processor, decoder_start_token_id):
        self.processor = processor
        self.decoder_start_token_id = decoder_start_token_id

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.decoder_start_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch


processor = WhisperProcessor.from_pretrained(base_model, language="English", task="transcribe")

model = WhisperForConditionalGeneration.from_pretrained(base_model)

model.generation_config.language = "English"
model.generation_config.task = "transcribe"

model.generation_config.forced_decoder_ids = None

data_collator = DataCollatorSpeechSeq2SeqWithPadding(
    processor=processor,
    decoder_start_token_id=model.config.decoder_start_token_id,
)

import evaluate

metric = evaluate.load("wer")


def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}


from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./model/training", 
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  
    learning_rate=1e-5,
    warmup_steps=500,
    max_steps=5000,
    gradient_checkpointing=True,
    fp16=True,
    eval_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False,
    push_to_hub=False,
)

from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=training_dataset,
    eval_dataset=testing_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

processor.save_pretrained(training_args.output_dir)

trainer.train()

# save the model
save_directory = "model/whisper-finetuned"
import os

os.makedirs(save_directory, exist_ok=True)

trainer.save_model(save_directory)

processor.save_pretrained(save_directory)


D:\Dev\Python\yapster\ml\venv\Lib\site-packages\accelerate\accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs
D:\Dev\Python\yapster\ml\venv\Lib\site-packages\torch\_dynamo\eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
D:\Dev\Python\yapster\ml\venv\Lib\site-packages\transformers\models\whisper\modeling_whisper.py:598: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\

Step,Training Loss,Validation Loss,Wer
1000,0.001200,0.278567,10.659694
2000,0.000400,0.307819,11.061947
3000,0.000200,0.324020,11.182623
4000,0.000100,0.334434,11.061947
5000,0.000100,0.339154,11.082060


You have passed task=transcribe, but also have set `forced_decoder_ids` to [[1, 50259], [2, 50359], [3, 50363]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=transcribe.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 448, 'suppress_tokens': [1, 2, 7, 8, 9, 10, 14, 25, 26, 27, 28, 29, 31, 58, 59, 60, 61, 62, 63, 90, 91, 92, 93, 359, 503, 522, 542, 873, 893, 902, 918, 922, 931, 1350, 1853, 1982, 2460, 2627, 3246, 3253, 3268, 3536, 3846, 3961, 

[]